In [0]:
%sql
-- Create SQL Procedure for sp_ADS_SMA_CPP_MAIN_PREPARE ETL Process
CREATE OR REPLACE PROCEDURE gap_catalog.ads_etl_owner.sp_ADS_SMA_CPP_MAIN_PREPARE_PART2(
    p_load_date STRING,
    p_process_key STRING
)
LANGUAGE SQL
 SQL SECURITY INVOKER
COMMENT 'ETL procedure for sp_ADS_SMA_CPP_MAIN_PREPARE'
AS
BEGIN
    -- Declare local variables
    DECLARE my_log_id STRING;
    BEGIN
        CALL gap_catalog.log.start_performance_test(
            test_case_name => 'ADS_SMA_PROC',
            step_name => 'ADS_SMA_CPP_MAIN_PREPARE_PART2',
            psource_sys_origin  => 'MAIN',
            process_key => p_process_key,
            target_table => 'STG_SMA_CASE_PHASE_PROPERTIES',
            log_id => my_log_id,
            p_load_date => p_load_date
        );

        TRUNCATE TABLE gap_catalog.ads_etl_owner.STG_SMA_CASE_PHASE_PROPERTIES;
        
        /* Fill Stage Table */
        insert  into  gap_catalog.ads_etl_owner.STG_SMA_CASE_PHASE_PROPERTIES
        select CPPROP_SOURCE_ID   
        , CPPROP_PARENT_SOURCE_ID 
        , CASE_SOURCE_ID     
        , CPPTP_SOURCE_ID   
        , ATTR_FP         
        , CASETYPE         
        , ATTR_VALUE    
        --,cpprop_value_text  
        , ATTR_BVALUE         
        , ATTR_TYPE           
        , STRUCTURETYPE           
        , MAPPING      
        , ct.ctp_key           
        from (select c.*, row_number() over (partition by cpprop_source_id order by timestamp desc) as rn
        from gap_catalog.ads_etl_owner.STG_SMA_CASE_PHASE_PROPERTIES_PARSE c) src
        join gap_catalog.ads_src_20250901.case_types ct on ct.ctp_source_id=src.casetype and ct.ctp_source_sys_origin='RDS_CASETYPE' and ct.ctp_valid_to=date'3000-01-01' 
        where rn=1;

        CALL gap_catalog.log.complete_performance_test(
            plog_id => my_log_id
        );

END;
    
END;